In [23]:
import csv
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
nltk.download('words')
from nltk.corpus import words
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer


with open("A1_dataset.csv", encoding="utf8", newline='') as file:
    data = csv.reader(file)
    next(data)
    data_list = list(data)
    
data_pre = []

for line in data_list:
    data_pre.append([line[0],line[1],line[2].lower()])


reg_exp = "((https?|www[\.][^\.\s])[^\s]+)" #Remove URLs
reg_exp1 = "[@]\w+"     #Remove Usernames
reg_exp2 = "&quot|&gt|&amp|&lt"  #Remove Special Words
reg_exp3= "[^a-zA-Z0-9\s]"  #Remove Special Characters
reg_exp4 = r"\b\w*\d\w*\b"  #Remove Numbers and Words Containing Numbers

#Remove Stopwords
stop_words = list(stopwords.words('english'))
extra_stop_words = ["u","ur","uk","go","x","xxx","l","r","b","c","d","e","f","g","h","j","k","m","n","o","p","q","v","w","y","z","im","imi"]
stop_words = stop_words + extra_stop_words
reg_exp5 = r"\b(" + r"|".join(stop_words) + r")\b"

reg_exp6 = "\s\s+"  #Remove Space

for i in range(len(data_pre)): 
    x = re.sub(reg_exp, "", data_pre[i][2])
    x = re.sub(reg_exp1, "", x)
    x = re.sub(reg_exp2, "", x)
    x = re.sub(reg_exp3, "", x)
    x = re.sub(reg_exp4, "", x)
    x = re.sub(reg_exp5, "", x)
    x = re.sub(reg_exp6, " ", x)
    y = x.split()   #Tokenisation
    data_pre[i][2] = y
        
#Spelling Correction
correct_words = words.words()

for i in range(len(data_pre)):
    for j in range(len(data_pre[i][2])):
        word = data_pre[i][2][j]
        temp = [(jaccard_distance(set(ngrams(word, 2)),set(ngrams(w, 2))),w)
                for w in correct_words if w[0]==word[0]]
        # print(sorted(temp, key = lambda val:val[0])[0][1])
        data_pre[i][2][j] = sorted(temp, key = lambda val:val[0])[0][1]

#Lemmatization
lem = WordNetLemmatizer()

for i in range(len(data_pre)):
    for j in range(len(data_pre[i][2])):
        word = data_pre[i][2][j]
        word = lem.lemmatize(word, wordnet.VERB)
        data_pre[i][2][j] = word
        # print(word) 
     

#Forming sentences
for i in range(len(data_pre)):
    data_pre[i][2] = " ".join(data_pre[i][2])

# print(data_pre)

# fields = ["LABEL","DATE_TIME","TEXT"]
# with open('Negative_Preprocessed.csv', 'w') as f:
      
#     # using csv.writer method from CSV package
#     write = csv.writer(f)
      
#     write.writerow(fields)
#     write.writerows(data_pre)

np.savetxt("Preprocessed.csv", data_pre, delimiter =",", fmt ='% s')



[nltk_data] Downloading package words to
[nltk_data]     C:\Users\charv\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
